### **Experimental Face Recognition**


In [ ]:
# import libraries
import numpy as np
import os
import cv2
import errno
import sys

# read image function
def read_image(image_path, image_size = None):
    c = 0
    X,y = [],[] # X is the image data, y is the label

    for dirname, dirnames, filenmaes in os.walk(image_path):# loop through all the files in the directory
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                try:
                    if(filename == ".directory"):
                        continue
                    filepath = os.path.join(subject_path, filename)
                    im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)

                    # resize the image to the desired size
                    if (image_size is not None):
                        im = cv2.resize(im, (200, 200))

                    # add the image data and the label to the X and y arrays
                    X.append(np.asarray(im, dtype=np.uint8))
                    y.append(c)

                # if there is an error, print it and continue to the next image
                except IOError as e:
                    print("I/O error({0}): {1}".format(e.errno, e.strerror))
                except:
                    print("Unexpected error:", sys.exc_info()[0])
                    raise
            c = c + 1



    # return the X and y arrays
    return [X,y]





In [ ]:
read_image(r"C:\Users\Lenovo\Downloads\dataset")


[[array([[218, 218, 219, ..., 167, 163, 165],
         [219, 219, 218, ..., 167, 164, 164],
         [219, 219, 218, ..., 167, 165, 165],
         ...,
         [116, 123, 122, ..., 189, 189, 190],
         [126, 127, 126, ..., 188, 188, 188],
         [120, 115, 116, ..., 186, 185, 186]], dtype=uint8),
  array([[178, 182, 172, ..., 173, 171, 170],
         [176, 176, 170, ..., 173, 174, 174],
         [174, 176, 178, ..., 172, 174, 175],
         ...,
         [146, 142, 135, ...,  95,  97,  90],
         [ 88,  93, 129, ...,  98,  98,  94],
         [ 96, 106, 136, ...,  99,  96,  95]], dtype=uint8),
  array([[173, 176, 175, ..., 233, 231, 230],
         [171, 173, 173, ..., 233, 232, 232],
         [170, 174, 175, ..., 233, 233, 233],
         ...,
         [ 83,  72,  63, ..., 161, 159, 158],
         [ 84,  81,  77, ..., 165, 163, 158],
         [ 89,  84,  79, ..., 161, 162, 162]], dtype=uint8),
  array([[180, 182, 180, ..., 180, 176, 183],
         [180, 183, 182, ..., 181, 174,

In [60]:
def face_rec(filepath, model):
    # names from the path of the image
    names = ['moises', 'marky']
    
    # Ensure images are resized to a consistent size
    sz = (200, 200)
    [X, y] = read_image(filepath, sz)

    if len(X) == 0 or len(y) == 0:
        print("No images found in the directory or unable to read images")
        return
    
    y = np.asarray(y, dtype=np.int32)

    # Train the model
    model.train(X, y)

    # Create a window to display the images
    camera = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Define the codec and create VideoWriter object
    frame_width = int(camera.get(3))
    frame_height = int(camera.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('OpenCV_Demo(Eigen).avi', fourcc, 20.0, (frame_width, frame_height))

    # Loop through the camera feed
    while True:
        ret, img = camera.read()
        if not ret:
            print("failed to grab frame")
            break

        faces = face_cascade.detectMultiScale(img, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(gray, sz, interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = names[params[0]]
                if (label == 'moises'):
                    cv2.putText(img, 'Recognized', (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else :
                    cv2.putText(img, "Not Recognized, ", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            except:
                continue

        cv2.imshow("camera", img)
        if cv2.waitKey(1) & 0xFF == ord("q") or cv2.getWindowProperty("camera", cv2.WND_PROP_VISIBLE) < 1:
            break

    camera.release()
    cv2.destroyAllWindows()



In [6]:
#model selection
eigenFR = cv2.face.EigenFaceRecognizer_create()
fisherFR = cv2.face.FisherFaceRecognizer_create()
lbphFR = cv2.face.LBPHFaceRecognizer_create()


In [62]:
face_rec("marcandme",eigenFR)

Face recognition Using sample video

In [3]:
import cv2
import numpy as np
import os

def read_image(image_path, image_size=None):
    c = 0
    X, y = [], []  # X is the image data, y is the label

    for dirname, dirnames, filenames in os.walk(image_path):  # loop through all the files in the directory
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                try:
                    if filename == ".directory":
                        continue
                    filepath = os.path.join(subject_path, filename)
                    im = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

                    # Resize the image to the desired size
                    if image_size is not None:
                        im = cv2.resize(im, (200, 200))

                    # Add the image data and the label to the X and y arrays
                    X.append(np.asarray(im, dtype=np.uint8))
                    y.append(c)
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
            c += 1

    return X, y

def recognize_faces_in_video(video_path, model):
    names = ['Moises', 'Nicole']  # Modify as needed
    sz = (200, 200)
    
    # Load face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Open the input video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return None
    
    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Create output video file
    output_path = os.path.splitext(video_path)[0] + "_output.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
        
        for (x, y, w, h) in faces:
            face_roi = gray_frame[y:y + h, x:x + w]
            face_resized = cv2.resize(face_roi, sz, interpolation=cv2.INTER_LINEAR)
            
            try:
                label, confidence = model.predict(face_resized)
                if label == names.index('moises'):
                    text = f"{names[label]}"
                    color = (0, 255, 0)  # Green for recognized faces
                else:
                    text = f"Not Recognized, {confidence:.2f}"
                    color = (0, 0, 255)  # Red for unrecognized faces
            except:
                text = "Error"
                color = (0, 0, 255)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
        out.write(frame)
    
    cap.release()
    out.release()
    
    return output_path


In [4]:
read_image(r"C:\Users\Lenovo\Downloads\dataset")

([array([[193, 192, 192, ..., 195, 195, 195],
         [192, 193, 194, ..., 196, 195, 194],
         [191, 193, 194, ..., 197, 196, 197],
         ...,
         [161, 164, 166, ..., 146, 144, 146],
         [159, 164, 166, ..., 147, 145, 149],
         [162, 164, 164, ..., 148, 145, 148]], dtype=uint8),
  array([[191, 192, 190, ..., 201, 202, 202],
         [194, 194, 193, ..., 203, 203, 201],
         [190, 190, 189, ..., 201, 201, 200],
         ...,
         [168, 169, 168, ..., 150, 149, 147],
         [167, 170, 171, ..., 151, 150, 148],
         [167, 167, 168, ..., 152, 151, 150]], dtype=uint8),
  array([[13, 13, 13, ..., 14, 15, 15],
         [13, 13, 13, ..., 14, 15, 15],
         [13, 13, 13, ..., 14, 15, 15],
         ...,
         [15, 14, 13, ..., 13, 13, 13],
         [13, 13, 12, ..., 13, 13, 13],
         [14, 14, 13, ..., 13, 13, 13]], dtype=uint8),
  array([[44, 44, 43, ..., 45, 47, 47],
         [42, 44, 45, ..., 46, 48, 49],
         [41, 43, 45, ..., 43, 44, 46],
 

In [7]:
recognize_faces_in_video(r"C:\Users\Lenovo\Downloads\dataset\Eigen_source_unrecognized.mp4", eigenFR)

KeyboardInterrupt: 

In [9]:
#using the read_images function to read the images from the directory and  the face_rec function to train the model and recognize the faces
#make a function to recognize the faces in the recorded video. 
# The output should return a video file with rectangles around the faces and the name of the person if recognized. 
# If not recognized, the output should return "Not Recognized" and the confidence level. 
# The function should take the path of the video file as an input and return the path of the output video file. 
# The function should also take the model as an input. The model can be either EigenFaceRecognizer, FisherFaceRecognizer, or LBPHFaceRecognizer. 
# The function should be able to handle any model and return the output video file. 
# The output video file should be saved in the same directory as the input video file. 
# The name of the output video file should be the same as the input video file with "_output" appended to it. 
# For example, if the input video file is "video.mp4", the output video file should be "video_output.mp4". 
# The function should return the path of the output video file.

def face_rec_video(filepath, model,filepath_video):
    # names from the path of the image
    names = ['Moises', 'Nicole']
    
    # Ensure images are resized to a consistent size
    sz = (200, 200)
    [X, y] = read_image(filepath, sz)

    if len(X) == 0 or len(y) == 0:
        print("No images found in the directory or unable to read images")
        return
    
    y = np.asarray(y, dtype=np.int32)

    # Train the model
    model.train(X, y)

    # Create a window to display the videos
    camera = cv2.VideoCapture(filepath_video)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # record the video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))
    
    # Loop through the camera feed
    while True:
        ret, img = camera.read()
        if not ret:
            print("failed to grab frame")
            break

        faces = face_cascade.detectMultiScale(img, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(gray, sz, interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = names[params[0]]
                if (label == 'Moises'):
                    cv2.putText(img, 'Certified Individual', (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else :
                    cv2.putText(img, "Not Recognized, " + str(params[1]).format(2), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            except:
                continue

        cv2.imshow("camera", img)
        if cv2.waitKey(1) & 0xFF == ord("q") or cv2.getWindowProperty("camera", cv2.WND_PROP_VISIBLE) < 1:
            break

    camera.release()
    cv2.destroyAllWindows()

In [10]:
face_rec_video("dataset",eigenFR,"Eigen_source_unrecognized.mp4")

failed to grab frame
